In [70]:
import warnings
warnings.filterwarnings(action='ignore')
from sunpy.net import Fido
from sunpy.net import attrs as a
import sunpy.map as m
import matplotlib.pyplot as plt
import astropy.units as u
from astropy.time import Time
import os
import glob as g
from PIL import Image

In [95]:
def make_gif(Start_Time, End_Time, Coronagraph, Cadence, path, Duration):
    """Downloading image data"""
    if Coronagraph == 'c2':
        result = Fido.search(a.Time(Start_Time, End_Time), a.Instrument.lasco, a.Detector.c2, a.Sample(Cadence*u.minute))
    else:
        result = Fido.search(a.Time(Start_Time, End_Time), a.Instrument.lasco, a.Detector.c3, a.Sample(Cadence*u.minute))
        
    file_path= path + '\\LASCO_raw_files'
    if not os.path.isdir(file_path):
        os.mkdir(file_path)

    data_path = file_path + '\\{file}'
    data = Fido.fetch(result,path= data_path)
    file = sorted(g.glob(file_path + '\\*.fts'))

    
    """Plotting images"""
    image_path = path + '\\LASCO_Images'
    if not os.path.isdir(image_path):
        os.mkdir(image_path)

    if Coronagraph == "c2":
        cmap = "soholasco2"
    else:
        cmap = "soholasco3"
        
    background_data=m.Map(file[0])
    if background_data.data.shape !=[1024,1024]:
        background_data = background_data.resample([512,512]*u.pixel)
        
    for i in range(1, len(file)):
        lasco=m.Map(file[i])
        date=lasco.meta['date-obs']
        time=lasco.meta['time-obs']
    
        if lasco.data.shape !=[1024,1024]:
            lasco=lasco.resample([512,512]*u.pixel)
            
        data=lasco.data-background_data.data
        image=m.Map(data,lasco.meta)
        date=image.meta['date-obs']
        time=image.meta['time-obs']

        fig=plt.figure(figsize=[5,5])
        image.cmap.set_bad('black')
        image.plot(annotate=False,clip_interval=[10,99.5]*u.percent, cmap=cmap)
        plt.text(30,30,'LASCO' + Coronagraph + date + ' ' + time,color= 'white',fontsize=14)
        image.draw_limb()
        plt.axis('off')
        plt.savefig(image_path + '\\lasco_{}.jpg'.format(i), dpi=300,bbox_inches='tight',pad_inches=0.0)
        plt.close(fig)


    """Creating GIF"""
    output_path = path + "\\gif"
    if not os.path.isdir(output_path):
        os.mkdir(output_path)
    image = sorted(g.glob(image_path + '\\*.jpg'))
    img = [Image.open(file) for file in image]
    img[0].save(output_path + "\\CME.gif", save_all=True, append_images=img[1:], duration=10, loop=0) 

    return print("\033[1mFind your gif in \033[0m" + output_path + "\033[1m  :D\033[0m")
    

In [97]:
Start_Time = input("Start Time :")
End_Time = input("End_Time:")
Coronagraph = input("c2 or c3? ").lower()
Cadence = int(input("Cadence of images in mins: "))
Path = input("Output path (do not include parenthesis): ")
Duration = input("Duration of gif: ")

Start Time : 2015/04/02 05:20
End_Time: 2015/04/02 06:38
c2 or c3?  c2
Cadence of images in mins:  24
Output path (do not include parenthesis):  C:\Users\ybaru\OneDrive\Desktop\trial
Duration of gif:  10


In [98]:
make_gif(Start_Time, End_Time, Coronagraph, Cadence, Path, Duration)

Files Downloaded:   0%|          | 0/4 [00:00<?, ?file/s]

INFO: Missing metadata for solar radius: assuming the standard radius of the photosphere. [sunpy.map.mapbase]
INFO: Missing metadata for solar radius: assuming the standard radius of the photosphere. [sunpy.map.mapbase]
INFO: Missing metadata for solar radius: assuming the standard radius of the photosphere. [sunpy.map.mapbase]
INFO: Missing metadata for solar radius: assuming the standard radius of the photosphere. [sunpy.map.mapbase]
INFO: Missing metadata for solar radius: assuming the standard radius of the photosphere. [sunpy.map.mapbase]
INFO: Missing metadata for solar radius: assuming the standard radius of the photosphere. [sunpy.map.mapbase]
INFO: Missing metadata for solar radius: assuming the standard radius of the photosphere. [sunpy.map.mapbase]
INFO: Missing metadata for solar radius: assuming the standard radius of the photosphere. [sunpy.map.mapbase]
INFO: Missing metadata for solar radius: assuming the standard radius of the photosphere. [sunpy.map.mapbase]
INFO: Miss